In [1]:
import numpy as np


def sigmoid(z):
    return 1 / (1 + np.exp(-z))


def activate(z):
    return np.tanh(z)


def dActivate(a):
    return 1 - np.square(a)


def cost(Yh, Y):
    """
    A -- numpy array of size (1, number of examples)
    Y -- label vector of size (1, number of examples)
    """
    return (np.dot(Y, np.log(Yh).T) + np.dot((1 - Y), np.log(1 - Yh).T))/-Y.shape[1]


def forward_propagate(W, b, Ap):
    Z = np.dot(W, Ap) + b
    A = activate(Z)
    
    n_layer = {'Z': Z, 'A': A}
    return n_layer


def output_backward_propagate(Y, A, Ap):
    m = Y.shape[1]
    
    dZ = A - Y
    dW = np.dot(dZ, Ap.T)/m
    db = np.sum(dZ, axis=1, keepdims = True)/m
    
    grad = {'dZ': dZ, 'dW': dW, 'db': db}
    return grad
    

def backward_propagate(A, Ap, dA):     
    m = A.shape[1]
    
    dZ = dA * dActivate(A)
    dW = np.dot(dZ, Ap.T)/m
    db = np.sum(dZ, axis=1, keepdims = True)/m
    
    grad = {'dZ': dZ, 'dW': dW, 'db': db}
    return grad


def nn_model(X, Y, hidden_nodes=(3, 3), learning_rate=0.01, iter=10000, print_th_iter=100):
    """
    X -- data of size (n, number of examples)
    Y -- true "label" vector of size (1, number of examples)
    """
    for i in range(iter):        
        for nl in hidden_nodes:
            W = np.random.randn() * 0.01
    

def logistic_regression(W, b, X, Y, learning_rate=0.01, iter=10000, print_th_iter=100):
    """
    W -- weights, a numpy array of size (n, 1)
    b -- bias, a scalar
    X -- data of size (n, number of examples)
    Y -- true "label" vector of size (1, number of examples)
    """
    
    for i in range(iter):
        grad, c = propagate(W, b, X, Y)
        
        W -= learning_rate * grad['dw']
        b -= learning_rate * grad['db']

        if (print_th_line > 0 and i % print_th_line == 0):
            print("Cost : {0}".format(c))
  
    features = {'W':W, 'b': b}
    
    return grad, features, c 


def accuracy(W, b, X, Y):
    p = predict(W, b, X)
    return 100 - np.sum(np.abs(p - Y))/Y.shape[1] * 100


In [2]:
training_set_X = np.random.randn(10, 5)
training_set_Y = np.array([[1, 0, 0, 1, 1]])
test_set_X = np.random.randn(10, 5)
test_set_Y = np.array([[1, 0, 0, 1, 1]])
W = np.zeros([10, 1])
b = np.zeros([1, 1])

grad, features, c = logistic_regression (W, b, training_set_X, training_set_Y, print_th_line=0)
W = features['W']
b = features['b']
print ("Cost for training set: {}".format(c))
print ("Cost for test set: {}".format(cost(predict(W, b, test_set_X), test_set_Y)))

print ("Accuracy of training: {}".format(accuracy(W, b, training_set_X, training_set_Y)))
print ("Accuracy of training: {}".format(accuracy(W, b, test_set_X, test_set_Y)))

Cost for training set: [[ 0.00089648]]
Cost for test set: [[ 2.53831288]]
Accuracy of training: 99.91041523235769
Accuracy of training: 53.65754757357009


In [3]:
import os 
import imageio
import matplotlib.pyplot as plt
import skimage.transform as imgxf

%matplotlib inline


def load_image(img_path, width, height):
    img = np.array(imageio.imread(img_path))
    return imgxf.resize(img, (width, height), mode='reflect').reshape((1, width*height*3)).T, img
    

def load_image_X(folder, width, height):
    fname = np.array([[f] for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]).T
    img_X = np.zeros((width*height*3, fname.shape[1]))
    it = np.nditer(fname, flags=['f_index'])
    while not it.finished:
        print("Loading : ", it[0])
        fpath = os.path.join(folder, str(it[0]))
        x, img = load_image(fpath, width, height)
        img_X[:, it.index] = x[:,0]
        it.iternext()
    
    return fname, img_X
    

num_px = 300
image_folder = "data/images/22-80"
image_path = image_folder + '/22-80-01.jpg'

files, image_X = load_image_X(image_folder, num_px, num_px)
print("Files: {}, image: {}".format(files.shape, image_X.shape))

FileNotFoundError: [Errno 2] No such file or directory: 'data/images/22-80'

In [4]:
import numpy as np

a = np.random.randn(1, 1)
b = np.random.randn(1, 3)
print(a)
print(b)
np.dot(a, b)

[[-1.73293708]]
[[ 0.40687633  0.68986766  1.471552  ]]


array([[-0.70509109, -1.19549725, -2.55010703]])